# Identify Super Voters
Extracts voter history data for a selected county (Wake) and tallies the number of time each DEM voter ("voter_reg_num") has voted, i.e. the number of counts. These voters are gjoi

In [1]:
import pandas as pd

In [136]:
pd.read_csv('./data/NCSBE/ncvhis_Statewide.txt',sep='\t',nrows=10,usecols=('county_desc','election_lbl','ncid')).columns

Index(['county_desc', 'election_lbl', 'ncid'], dtype='object')

In [64]:
#Read file 
#dfHistWake = pd.read_csv('./data/NCSBE/ncvhis_Wake.csv')
df = pd.read_csv('./scratch/wake_history.csv',usecols=('county_desc','election_lbl','ncid'))
df.head()

,county_desc,election_lbl,ncid
0,WAKE,11/06/2018,AA100110
1,WAKE,11/04/2014,AA100110
2,WAKE,11/08/2016,AA100110
3,WAKE,11/06/2012,AA100110
4,WAKE,11/02/2010,AA100110


In [65]:
#Subset for meaningful elections
elections = ('10/10/2017','11/07/2017','11/06/2018','11/08/2016','11/06/2012')
dfSubset = df.loc[df.election_lbl.isin(elections),:]

In [100]:
dfPivot = pd.pivot_table(dfSubset,
                         columns = 'election_lbl',
                         index = 'ncid',
                         aggfunc = 'count',
                         fill_value = 0
                        ).droplevel(level=0,axis=1)

In [101]:
#Rename columns
dfPivot.rename({'10/10/2017':'Oct17',
                '11/06/2012':'Nov12',
                '11/06/2018':'Nov18',
                '11/07/2017':'Nov17',
                '11/08/2016':'Nov16'},
               axis=1,inplace=True)

In [124]:
#Filters
e12 = dfPivot.Nov12 == 1
e16 = dfPivot.Nov16 == 1
e17 = (dfPivot.Oct17 == 1) | (dfPivot.Nov17 == 1)
e18 = dfPivot.Nov18 == 1

In [128]:
dfPivot["MECE"] = -1
dfPivot.loc[e17, "MECE"] = 1
dfPivot.loc[~e17 & e18, "MECE"] = 2
dfPivot.loc[~e17 & ~e18 & e16, "MECE"] = 3
dfPivot.loc[~e17 & ~e18 & ~e16 & e12, "MECE"] = 4
dfPivot.loc[~e17 & ~e18 & ~e16 & ~e12, "MECE"] = 5
dfPivot.head()

election_lbl,Oct17,Nov12,Nov18,Nov17,Nov16,MECE
ncid,,,,,,
AA100110,0,1,1,0,1,2
AA100458,0,1,0,0,1,3
AA100876,0,1,0,0,0,4
AA100921,0,0,0,0,1,3
AA101032,1,1,1,1,1,1


In [129]:
dfPivot[dfPivot.MECE < 0]

election_lbl,Oct17,Nov12,Nov18,Nov17,Nov16,MECE
ncid,,,,,,


In [ ]:
dfWakeDems = dfHistWake[dfHistWake.voted_party_cd == 'DEM']

In [ ]:
dfVoteCount = pd.DataFrame(dfWakeDems['voter_reg_num'].value_counts()).reset_index()
dfVoteCount.head()

In [ ]:
dfVoteGeo = pd.read_csv('./data/NCSBE/ncvoter_Wake_geo.csv')
dfVoteGeo2 = pd.merge(dfVoteGeo,dfVoteCount,how = 'right',left_on='voter_reg_num',right_on='index')

In [ ]:
dfVoteGeo2.to_csv('./scratch/ncvoter_Wake_geo_count.csv')